In [ ]:
# FM
# input_dim, factors_num
# field_dims, embed_dim=16

input_dim = len(X_train[0])
embedding_size = 768
field_num = len(columns_to_train)

# 先跑最好的（最新的）
# dl_learning_rate_list = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05]
dl_learning_rate_list = [0.0001, 0.001, 0.01] # (4) 1e-3(0.001) 先跑看看，接下來測試
print("dl_learning_rate_list:", len(dl_learning_rate_list))
epochs_num_list = [10, 20, 30] # (2) [50, 100]，先跑 100，邊跑邊畫圖，還沒收斂繼續加 跑一個畫圖 接下來測試 還有 optimizer
print("epochs_num_list:", len(epochs_num_list))
batch_size_list = [64, 128, 256, 512] # [256, 512] # (1) 512，一定要 shuffle，一開始要決定好
print("batch_size_list:", len(batch_size_list))
mlp_dims_list = [tuple([size] * num_layers) for num_layers in range(1, 4) for size in [i*100 for i in range(1, 5)]]
print("mlp_dims_list:", len(mlp_dims_list)) # 先跑 3 層 256
mlp_dims_list_2times = [tuple([size] * num_layers) for num_layers in range(1, 4) for size in [2**i for i in range(5, 10)]]
print("mlp_dims_list_2times:", len(mlp_dims_list_2times)) # 
dropout_list_1 = [size * 0.1 for size in range(1, 6)]
print("dropout_list_1:", len(dropout_list_1)) # 0, 0.5 # 一開始要決定好 (3) 先 0
dropout_list_2 = [tuple([size*0.1] * 2) for size in range(1, 6)]
print("dropout_list_2:", len(dropout_list_2))
dropout_list_3 = [tuple([size*0.1] * 3) for size in range(1, 6)]
print("dropout_list_3:", len(dropout_list_3))

FM_params = {
    'hyperparameters': {
        'input_dim': [input_dim], 
#         'factors_num': [5, 10, 15],
#         'factors_num': [i+1 for i in range(20)]
#         'factors_num': [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
        'factors_num': [2**i for i in range(3, 7)]
    },
    'func_params':{
        'task_type': ['Regression'], 'loss_type': ['RMSE'], 'optimizer_type': ['Adam'], 
#         'dl_learning_rate': [0.01, 0.02, 0.001], 
        'dl_learning_rate': dl_learning_rate_list,
        'epochs_num': epochs_num_list, 
#         'batch_size': [100, 500, 1000]
        'batch_size': batch_size_list
    }
}

# MLP
# input_dim, embed_dims, dropout

MLP_params = {
    'hyperparameters': {
        'input_dim': [input_dim], 
#         'factors_num': [5, 10, 15],
        'embed_dims': [tuple([size] * num_layers) for num_layers in range(1, 4) for size in [i*100 for i in range(1, 5)]],
        'dropout': dropout_list_1
    },
    'func_params':{
        'task_type': ['Regression'], 'loss_type': ['RMSE'], 'optimizer_type': ['Adam'], 
#         'dl_learning_rate': [0.01, 0.02, 0.001], 
        'dl_learning_rate': dl_learning_rate_list,
        'epochs_num': epochs_num_list, 
#         'batch_size': [100, 500, 1000]
        'batch_size': batch_size_list
    }
}

# DeepFM
# feature_sizes, embedding_size, hidden_dims, num_classes=1, dropout=[0.5, 0.5]

# Paper:
# test activation function: relu, tanh (relu is better)
# embedding size seems to be 5
# dropout: test 1, 0.9, 0.8, 0.7, 0.6, 0.5 (when 0.6 ~ 0.9 is better)
# hidden layer num: test 1, 3, 5, 7, better at begining, but more getting worse
# shape: given layer num and total neuron num, test 4 shape, such as 3, 600 =>
# constant (200-200-200), increasing (100-200-300), decreasing (300-200-100), and diamond (150-300-150)
# constant is better, which is consistent with previous studies

DeepFM_params = {
    'hyperparameters': {
        'feature_sizes': [[embedding_size for i in range(field_num)]], 
#         'embedding_size': [4, 8], 
        'embedding_size': [5, 10],
#         'hidden_dims': [[64, 32], [128, 64], [256, 128]], 
#         'hidden_dims': [[100, 100, 100], [200, 200, 200], [400, 400, 400]],
        'hidden_dims': mlp_dims_list,
        'num_classes': [1], 
#         'dropout': [(0.5, 0.5)]
        'dropout': dropout_list_2
    },
    'func_params':{
        'task_type': ['Regression'], 'loss_type': ['RMSE'], 'optimizer_type': ['Adam'], 
#         'dl_learning_rate': [0.01, 0.02, 0.001], 
        'dl_learning_rate': dl_learning_rate_list,
        'epochs_num': epochs_num_list, 
#         'batch_size': [100, 500, 1000]
        'batch_size': batch_size_list
    }
}

# AFM
# field_dims, embed_dim, attn_size, dropouts 
# embed_dim=16, attn_size=16, dropouts=(0.2, 0.2)

# Paper
# optimizer: mini-batch Adagrad. The batch size for Frappe and MovieLens is set to 128 and 4096 
# The embedding size is set to 256 for all methods
# without special mention, the attention factor is also 256 (test on 1, 4, 8, 16, 32, 64, 128, 256)
# Validation error: performance is stable across different size of attention factors
# also test dropout for fm, afm, fifm, 0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8
# Specifically, for AFM, the optimal dropout ratio on Frappe and MovieLens is 0.2 and 0.5
# also test l2 regulation(attention network) from 0, 0.5, 1, 2, 4, 8, 16
# prove using l2 is better, and when l2 is larger, rmse is decreasing gently 

AFM_params = {
    'hyperparameters': {
        'field_dims': [[embedding_size for i in range(field_num)]], 
        'embed_dim': [768], 
#         'attn_size': [8, 16], 
        'attn_size': [1, 4, 8, 16, 32, 64, 128, 256],
#         'dropouts': [(0.5, 0.5)]
#         'dropouts': [(0.2, 0.2), (0.5, 0.5)]
        'dropouts': dropout_list_2
    },
    'func_params':{
        'task_type': ['Regression'], 'loss_type': ['RMSE'], 'optimizer_type': ['Adam'], 
#         'dl_learning_rate': [0.01, 0.02, 0.001], 
        'dl_learning_rate': dl_learning_rate_list,
        'epochs_num': epochs_num_list, 
#         'batch_size': [100, 500, 1000]
        'batch_size': batch_size_list
    }
}

# DCN
# field_dims, embed_dim, num_layers, mlp_dims, dropout 
# embed_dim=16, num_layers=3, mlp_dims=(16, 16), dropout=0.2

# Paper:
# cross layer(num_layers): 1~6, hidden layer: 2 ~ 5, with size 32 ~ 1024, 
# Based on test loss, 2 deep layers of size 1024 and 6 cross layers for the DCN model
# 5 deep layers of size 1024 for the DNN
# Based on validation loss, more cross layer num DCN loss decrease
 
# initial learning rate was tuned from 0.0001 to 0.001 with increments of 0.0001
# early stopping: at training step 150,000
# optimization: mini-batch stochastic optimization with Adam optimizer. 
# batch size: 512. Batch normalization was applied to the deep network
# gradient clip: 100
# Real-valued features are normalized by applying a log transform. 
# For categorical features, we embed the features in dense vectors of dimension 6×(category cardinality)1/4.
# Concatenating all embeddings results in a vector of dimension 1026

# Not CTR task, grid search:
# deep layers num ranged from 1 to 10 with layer size from 50 to 300. 
# The number of cross layers ranged from 4 to 10. 
# The number of residual units ranged from 1 to 5 with their input dim and cross dim from 50 to 300.
# For DCN, the input vector was fed to the cross network directly

DCN_params = {
    'hyperparameters': {
        'field_dims': [[embedding_size for i in range(field_num)]], 
        'embed_dim': [768], 
#         'num_layers': [3],
        'num_layers': [i+1 for i in range(6)],
#         'mlp_dims': [(16, 16)],
#         'mlp_dims': mlp_dims_list_2times,
        'mlp_dims': mlp_dims_list,
#         'dropout': [0.2, 0.5]
        'dropout': dropout_list_1
    },
    'func_params':{
        'task_type': ['Regression'], 'loss_type': ['RMSE'], 'optimizer_type': ['Adam'], 
#         'dl_learning_rate': [0.01, 0.02, 0.001], 
        'dl_learning_rate': dl_learning_rate_list,
        'epochs_num': epochs_num_list, 
#         'batch_size': [100, 500, 1000]
        'batch_size': batch_size_list
    }
}

# xDeepFM
# embed_dim, mlp_dims, dropout, cross_layer_sizes, split_half=True 
# embed_dim=16, cross_layer_sizes=(16, 16), split_half=False, mlp_dims=(16, 16), dropout=0.2

# Paper
# Learning rate : 0.001. optimization method: Adam, mini-batch size: 4096. 
# L2 regularization with λ = 0.0001 for DNN, DCN, Wide&Deep, DeepFM and xDeepFM, dropout 0.5 for PNN
# embed_dim = 10, 
# Neuron num per layer: (1) 400 for DNN layers; 
# (2) 200 for CIN layers on Criteo dataset, and 100 for CIN layers on Dianping and Bing News datasets
# test cin num_layer 1, 2, 3, 4 (3 best, degrad after 3)
# test cin num neurons 20, 40, 100, 200, Bing news is increasing(200 best), Dianping is 20 ~ 100 better, 200 degrades 
# test cin sigmoid, tanh, relu, identity 4 activation, identity is best, then relu
# best cross depth and dnn depth is (3, 2)

xDeepFM_params = {
    'hyperparameters': {
        'field_dims': [[embedding_size for i in range(field_num)]], 
        'embed_dim': [768], 
#         'cross_layer_sizes': [(16, 16)],
        'cross_layer_sizes': [tuple([neuron] * layer) for layer in range(1, 4) for neuron in [20, 40, 60]],
        "split_half": [False],
#         'mlp_dims': [(16, 16)],
        'mlp_dims': mlp_dims_list,
#         'dropout': [0.2, 0.5]
        'dropout': dropout_list_1
    },
    'func_params':{
        'task_type': ['Regression'], 'loss_type': ['RMSE'], 'optimizer_type': ['Adam'], 
#         'dl_learning_rate': [0.01, 0.02, 0.001], 
        'dl_learning_rate': dl_learning_rate_list,
        'epochs_num': epochs_num_list, 
#         'batch_size': [100, 500, 1000]
        'batch_size': batch_size_list
    }
}

# AutoInt
# field_dims, embed_dim, atten_embed_dim, num_heads, num_layers, mlp_dims, dropouts, has_residual=True
# embed_dim=16, atten_embed_dim=64, num_heads=2, num_layers=3, mlp_dims=(400, 400), dropouts=(0, 0, 0)

# Paper:
# embed_dim is set to 16, batch size set to 1024
# num_head is 2, num_layer(interaction layer) is 3, num of hidden units (interaction layer) is 32
# test dropout from 0.1 ~ 0.9
# optimizer is Adam
# test num_layer(interaction layer) from 0 ~ 4, 1 increase dramaticaly, 1 ~ 4 become stable
# test atten_embed_dim 8, 16, 24, 32, movie len is getting better, for KDD12, 24 is best, then decrease
# hidden units shape is set to (1, 200) or (4, 100)
# residaul is crucial

AutoInt_params = {
    'hyperparameters': {
        'field_dims': [[embedding_size for i in range(field_num)]], 
        'embed_dim': [768], 
#         'atten_embed_dim': [(64, 32)],
#         'atten_embed_dim': [2**i for i in range(3, 6)],
        'atten_embed_dim': [8, 16, 24],
        "num_heads": [2],
#         "num_layers": [3],
        "num_layers": [i+1 for i in range(3)],
#         'mlp_dims': [(16, 16), (400, 400)],
        'mlp_dims': mlp_dims_list,
#         'dropouts': [(0.5, 0.5, 0.5)],
        'dropouts': dropout_list_3,
        "has_residual": [True]
    },
    'func_params':{
        'task_type': ['Regression'], 'loss_type': ['RMSE'], 'optimizer_type': ['Adam'], 
#         'dl_learning_rate': [0.01, 0.02, 0.001], 
        'dl_learning_rate': dl_learning_rate_list,
        'epochs_num': epochs_num_list, 
#         'batch_size': [100, 500, 1000]
        'batch_size': batch_size_list
    }
}

# AFN
# field_dims, embed_dim, LNN_dim, mlp_dims, dropouts
# embed_dim=16, LNN_dim=1500, mlp_dims=(400, 400, 400), dropouts=(0, 0, 0)

# Paper:
# LNN_dim: 1500, 1200, 800, 600, embed_dim: 10
# batch size: 4096
# optimizer: Adam # earning rate :0.001
# mlp_dims: (400, 400, 400)
# All the other hyperparameters are tuned on the validation set

AFN_params = {
    'hyperparameters': {
        'field_dims': [[embedding_size for i in range(field_num)]], 
        'embed_dim': [768], 
#         'LNN_dim': [1500, 1000],
        'LNN_dim': [1500, 1200, 1000, 800, 600, 400, 200],
#         'mlp_dims': [(400, 400, 400)],
        'mlp_dims': mlp_dims_list,
#         'dropouts': [(0.5, 0.5, 0.5)]
        'dropouts': dropout_list_3
    },
    'func_params':{
        'task_type': ['Regression'], 'loss_type': ['RMSE'], 'optimizer_type': ['Adam'], 
#         'dl_learning_rate': [0.01, 0.02, 0.001], 
        'dl_learning_rate': dl_learning_rate_list,
        'epochs_num': epochs_num_list, 
#         'batch_size': [100, 200, 500]
        'batch_size': batch_size_list
    }
}

# # Linear parameters
Linear_params = {
    'hyperparameters': {
        
    },
    'func_params':{
        'task_type': ['Regression'], 'loss_type': [None], 'optimizer_type': [None], 
        'dl_learning_rate': [None], 'epochs_num': [None], 'batch_size': [None]
    }
}

# # K-Nearest Neighbors parameters
# n_neighbors: int, default=5  # Number of neighbors
# weights: {'uniform', 'distance'}, default='uniform'  # Weight function used in prediction
# metric: {'euclidean', 'manhattan', 'chebyshev', 'minkowski', 
#          'wminkowski', 'seuclidean', 'mahalanobis'}, default='minkowski'  # Distance metric

KNN_params = {
    'hyperparameters': {
#         'n_neighbors': [5],
        'n_neighbors': [3, 5, 7, 9],
#         'weights': ['uniform'],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean']
    },
    'func_params':{
        'task_type': ['Regression'], 'loss_type': [None], 'optimizer_type': [None], 
        'dl_learning_rate': [None], 'epochs_num': [None], 'batch_size': [None]
    }
}

# # Support Vector Machine parameters
# C: float, default=1.0  # Penalty parameter C of the error term
# kernel: {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'}, default='rbf'  # Kernel function
# gamma: {'scale', 'auto'} or float, default='scale'  # Kernel coefficient for 'rbf', 'poly' and 'sigmoid'

SVM_params = {
    'hyperparameters': {
#         'C': [1.0],
        'C': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0], 
#         'kernel': ['rbf', 'poly', 'sigmoid'],
        'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
#         'gamma': ['scale']
        'gamma': ['scale', 'auto']
    },
    'func_params':{
        'task_type': ['Regression'], 'loss_type': [None], 'optimizer_type': [None], 
        'dl_learning_rate': [None], 'epochs_num': [None], 'batch_size': [None]
    }
}

# # Decision Tree parameters
# max_depth: int or None, default=None  # Maximum depth of the tree
# min_samples_split: int or float, default=2  # Minimum number of samples required to split an internal node
# min_samples_leaf: int or float, default=1  # Minimum number of samples required to be at a leaf node

DecisionTree_params = {
    'hyperparameters': {
#         'max_depth': [None],
#         'min_samples_split': [2],
#         'min_samples_leaf': [1]
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'func_params':{
        'task_type': ['Regression'], 'loss_type': [None], 'optimizer_type': [None], 
        'dl_learning_rate': [None], 'epochs_num': [None], 'batch_size': [None]
    }
}

# # Random Forest parameters
# max_depth: int or None, default=None  # Maximum depth of the tree
# min_samples_split: int or float, default=2  # Minimum number of samples required to split an internal node
# min_samples_leaf: int or float, default=1  # Minimum number of samples required to be at a leaf node
# n_estimators: int, default=100  # Number of trees in the forest

RandomForest_params = {
    'hyperparameters': {
#         'max_depth': [None],
#         'min_samples_split': [2],
#         'min_samples_leaf': [1],
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10],
        'min_samples_leaf': [1, 2, 4],
#         'n_estimators': [50, 100]
        'n_estimators': [10, 20, 30 ,40 ,50, 60, 70, 80, 90, 100]
    },
    'func_params':{
        'task_type': ['Regression'], 'loss_type': [None], 'optimizer_type': [None], 
        'dl_learning_rate': [None], 'epochs_num': [None], 'batch_size': [None]
    }
}

# # AdaBoost parameters
# n_estimators: int, default=50  # Maximum number of estimators at which boosting is terminated
# learning_rate: float, default=1.0  # Learning rate shrinks the contribution of each classifier

AdaBoost_params = {
    'hyperparameters': {
#         'n_estimators': [50, 100],
        'n_estimators': [10, 20, 30 ,40 ,50, 60, 70, 80, 90, 100],
#         'learning_rate': [0.01, 0.02, 0.001]
        'learning_rate': [i/100 for i in range(1, 21)],
    },
    'func_params':{
        'task_type': ['Regression'], 'loss_type': [None], 'optimizer_type': [None], 
        'dl_learning_rate': [None], 'epochs_num': [None], 'batch_size': [None]
    }
}

# # XGBoost parameters
# learning_rate: float, default=0.1  # Boosting learning rate (xgb's "eta")
# n_estimators: int, default=100  # Number of boosted trees to fit
# subsample: float, default=1  # Subsample ratio of the training instances (xgb's "subsample")
# colsample_bytree: float, default=1  # Subsample ratio of columns when constructing each tree (xgb's "colsample_bytree")

XGBoost_params = {
    'hyperparameters': {
#         'learning_rate': [0.01, 0.02, 0.001],
        'learning_rate': [i/100 for i in range(1, 11)],
#         'n_estimators': [50, 100],
        'n_estimators': [20, 40, 60, 80, 100],
#         'subsample': [1],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
#         'colsample_bytree': [1]
        'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    },
    'func_params':{
        'task_type': ['Regression'], 'loss_type': [None], 'optimizer_type': [None], 
        'dl_learning_rate': [None], 'epochs_num': [None], 'batch_size': [None]
    }
}

# Training hyperparameter combinations setting

func_params_list = {
    
    "FM": FM_params,
    "MLP": MLP_params,
    "DeepFM": DeepFM_params,
    "AFM": AFM_params,
    "DCN": DCN_params,
    "xDeepFM": xDeepFM_params,
    "AutoInt": AutoInt_params,
    "AFN": AFN_params,
    
    "Linear": Linear_params,
    "KNN": KNN_params,
    "SVM": SVM_params,
    "DecisionTree": DecisionTree_params,
    "RandomForest": RandomForest_params,
    "AdaBoost": AdaBoost_params,
    "XGBoost": XGBoost_params 
    
}